# Standard Bayesian Weight Analyzer Clustering Solver

This Jupyter Notebook provides a solver for the Weight Analyzer method using standard preference data and supports clustering of decision makers. The notebook guides you through the process of inputting your preference matrix, specifying the number of clusters, running the Weight Analyzer algorithm with clustering, and visualizing the results.

**Instructions for Use:**
- Each code cell can be executed independently.
- To run a cell, select it and press `Shift+Enter`. This will execute the cell and display the output below it.
- Follow the workflow from top to bottom, ensuring that variables and modules are defined before they are used in subsequent cells.

Use this notebook to analyze your multi-criteria decision-making problems with the Weight Analyzer approach, leveraging the provided visualizations and analysis tools.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../../src"))

# Import Required Libraries
import numpy as np
from BayesMCDM import WeightAnalyzer
from BayesMCDM.Visualizer import CredalRanking, WeightVisualizer

# Bayesian Weight Analyzer with Clustering

The Weight Analyzer method is a Multi-Criteria Decision-Making (MCDM) technique for determining the relative importance (weights) of a set of criteria. In Weight Analyzer, the decision-maker provides a preference matrix that expresses the relative preference of one criterion over another.

For group decision-making, the responses from all participants are aggregated into a preference array. This array serves as the input for the Bayesian Weight Analyzer, which then computes the optimal weight distributions for each criterion, reflecting the collective preferences of the group.

There is also a variable named `criteria_names`, which contains the names of the criteria in the same order as in the matrices. This variable will be used later for visualization, ensuring that plots and charts are clearly labeled with the appropriate criterion names.

**If you want to solve your own problem**, you only need to change the values of `preferences` to reflect your own data, set the number of clusters in `cluster_no`, and then execute the rest of the cells in this notebook.

In [ ]:
# Define the preferences array
weights = [
    [0.0811, 0.1081, 0.1622, 0.027,  0.1351, 0.0541, 0.2432, 0.1892],
    [0.0263, 0.0526, 0.2105, 0.1053, 0.1316, 0.0789, 0.2368, 0.1579],
    [0.0571, 0.0571, 0.0857, 0.0286, 0.1429, 0.1429, 0.2571, 0.2286],
    [0.0488, 0.0244, 0.1951, 0.0488, 0.2195, 0.0732, 0.1951, 0.1951],
    [0.0606, 0.1212, 0.2727, 0.0303, 0.1212, 0.0909, 0.1515, 0.1515],
    [0.0333, 0.0667, 0.3,    0.0333, 0.1,    0.1667, 0.1667, 0.1333]
]

criteria_names = ["C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8"]
cluster_no = 2  # Set the number of clusters for decision makers

## Solving the Bayesian Weight Analyzer with Clustering

In this section, we solve the Bayesian Weight Analyzer method using the provided `preferences` array and the specified number of clusters. Each row in the array represents the responses from a decision maker, and each column corresponds to a criterion.

The Bayesian Weight Analyzer algorithm samples from the posterior distributions of the criteria weights, capturing the uncertainty and variability in the group’s preferences. This probabilistic approach offers a deeper and more nuanced understanding of the relative importance of each criterion, compared to traditional deterministic methods.

In [ ]:
# Initialize the StandardSWING model with clustering
weight_analyzer = WeightAnalyzer.StandardWeightAnalyzer(weights=weights, dm_clusterno=cluster_no)
weight_analyzer.sampling()

## Summary Statistics
In the `weight_analyzer` object, several statistics are available from the posterior distribution computed by the Bayesian BWM algorithm:

- **`weight_analyzer.aggregated_weight`**: The mean of the aggregated posterior weight distribution for each criterion, representing the overall group consensus.
- **`weight_analyzer.aggregated_weight_samples`**: The full set of posterior samples for the aggregated weights, capturing the uncertainty and variability in the group’s preferences.

The two attributes summarize the aggregated (group-level) results.

In [ ]:
print("the mean of aggregated posterior weights for each criterion (group consensus):")
for name, weight in zip(criteria_names, weight_analyzer.aggregated_weight):
    print(f"{name}: {weight:.4f}")

# Visualization of Bayesian Weight Analyzer Results

- **Credal Ranking:**  
    The credal ranking visualizes the probability that one criterion is more important than another, based on the posterior distributions of their weights. This approach quantifies the strength of preference between criteria, providing a probabilistic ranking rather than a single deterministic order.  
    Multiple plots are provided for credal ranking, and you can use any of them in your report according to your preference. The plots are customizable—refer to the documentation or other visualization pages for more details on available options.

- **Weight Distributions:**  
    Since the Bayesian Weight Analyzer produces a distribution for each criterion's weight (rather than a single value), we can visualize these distributions to understand the uncertainty and variability in the estimated importance of each criterion. Several types of weight distribution plots are available, and you may choose whichever best fits your reporting needs. These plots are also customizable; see the visualization documentation for further customization options.

## Credal Ranking 
For credal ranking visualization, it is necessary to provide the samples from the posterior distribution of the criteria weights, along with the `criteria_names` for labeling. In this example, we use the aggregated posterior weight samples (`weight_analyzer.aggregated_weight_samples.T`) and the corresponding `criteria_names` to plot the credal ranking, which reflects the group-level consensus on the relative importance of each criterion.

The `CredalRanking` object contains a variable `probs`, which is a matrix of computed probabilities. Each entry `probs[i, j]` represents the probability that criterion *i* is more important than criterion *j* according to the posterior samples.

In [ ]:
credal_ranking = CredalRanking(weight_analyzer.aggregated_weight_samples, criteria_names=criteria_names)
# Print the credal ranking probability matrix as a DataFrame for better readability
print(credal_ranking.credal_probs())

### Default Plot 
By calling the `plot` function from the `credal_ranking` object, we generate a weighted directed graph that visually represents the probabilistic ranking of the criteria. In this graph, each node corresponds to a criterion, and a directed edge from node A to node B (A → B) indicates that criterion A is more likely to be more important than criterion B based on the posterior distributions.

There are two ways to display the probabilities on the graph:

- **If `show_probs` is `False`**: The probabilities are encoded by the line style of the edges, as indicated in the legend. Different line styles represent different probability ranges.
- **If `show_probs` is `True`**: The actual probability values are displayed as weights on the edges, providing a quantitative measure of confidence for each pairwise comparison.

This visualization helps to intuitively understand both the ranking and the strength of preference among the criteria.

In [ ]:
credal_ranking.plot(show_probs=False)

### Ranking plot 
The `plot_ranking` function creates a simpler graph to show the ranking of criteria. It arranges the criteria from most important (at the top) to least important (at the bottom). If a criterion is above another and there is no edge between them, it means the top one is more important with 100% certainty. Edges are only drawn when the probability is less than 1, and the probability value is shown on the edge. This makes the plot easier to read and helps you quickly see where there is uncertainty in the ranking.

The `plot_ranking` function has several parameters to customize the appearance of the plot:

- **graph_layout**: Sets the direction of the graph. Use `"TD"` for top-down (most important at the top) or `"LR"` for left-to-right (most important on the left).
- **node_fontsize**: Sets the font size for the criterion names (nodes).
- **node_boldfont**: If `True`, makes the node labels bold.
- **edge_fontsize**: Sets the font size for the probability labels on the edges.
- **edge_boldfont**: If `True`, makes the edge labels bold.

These options help you adjust the style and orientation of the ranking plot for better readability.

In [ ]:
credal_ranking.plot_ranking(graph_layout="TD", node_fontsize=16, node_boldfont=True, edge_fontsize=16, edge_boldfont=True)

## Weight Distribution
In Bayesian Weight Analyzer, each criterion's weight is represented by a distribution rather than a single value, reflecting the uncertainty in the group’s preferences. To visualize these distributions, we use the `WeightVisualizer` class, which requires the posterior weight samples and the corresponding `criteria_names` as input. In this example, we use the aggregated posterior weight samples (`weight_analyzer.aggregated_weight_samples.T`) to generate several types of weight distribution plots. These visualizations help us understand both the central tendency and the variability of the estimated importance for each criterion.

In [ ]:
weight_visualizer = WeightVisualizer(weight_analyzer.aggregated_weight_samples, criteria_names=criteria_names)

### Histogram of Weight Samples: `plot_distributions`

The first plot, generated by the `plot_distributions` method, displays the histogram of the posterior weight samples for each criterion. This visualization shows the distribution of possible weight values, reflecting the uncertainty in the estimated importance of each criterion. The **red dashed line** in each subplot marks the mean (average) of the distribution, providing a clear reference for the central tendency of the weights.

**Customizable Parameters:**

- `ncols=3`: Sets the number of columns in the grid layout of the subplots. For example, with 8 criteria and `ncols=3`, the plots will be arranged in 3 columns and 3 rows.
- `font_size=12`: Specifies the font size for axis labels and titles within each subplot, improving readability.
- `font_bold=False`: If set to `True`, makes the font bold for all text in the plots; otherwise, regular font weight is used.
- `xlim=[0, 0.4]`: Defines the range of the x-axis for all histograms, ensuring consistent scaling and easier comparison across criteria.

These parameters allow you to tailor the appearance and layout of the histogram plots to best fit your reporting or presentation needs.

In [ ]:
weight_visualizer.plot_distributions(ncols=3, font_size=12, font_bold=False, xlim=[0,0.4])

### Ridge Plot of Weight Distributions

The `ridge_plot` method in the `WeightVisualizer` class displays the posterior weight distributions for all criteria as overlapping density curves, stacked vertically. This visualization, also known as a "joyplot," allows for easy comparison of the shape, spread, and central tendency of each criterion's weight distribution.

**Key Features:**
- Each curve represents the posterior distribution of a criterion’s weight.
- Criteria are labeled on the y-axis for clarity.
- The plot highlights differences in uncertainty and the relative importance of each criterion.
- The `font_bold` parameter can be set to `True` to make axis labels and titles bold for improved readability.

This plot is especially useful for visually comparing the uncertainty and ranking of criteria in Bayesian Weight Analyzer analyses.

In [ ]:
weight_visualizer.ridge_plot(font_bold=True)

## Jackplot of Weight Distributions

The `vertical_jakplot` method in the `WeightVisualizer` class provides a "jackplot" visualization, where the x-axis represents the criteria and the y-axis shows the distribution of weights for each criterion, estimated using a kernel density estimator (KDE). This plot allows you to compare the uncertainty and central tendency of the weights across all criteria in a single, compact figure.

- **Blue dots**: Indicate the mean (average) of each criterion's weight distribution, providing a quick visual reference for the central value.
- **KDE curves**: Show the shape and spread of the posterior weight distributions for each criterion.

**Customizable Parameters:**
- `ylim`: Sets the limits of the y-axis (e.g., `ylim=[0, 0.4]`) to control the visible range of weight values.
- `font_size`: Adjusts the font size for axis labels and tick marks.
- `font_bold`: If `True`, makes all text in the plot bold for improved readability.

This visualization is useful for quickly assessing both the ranking and the uncertainty of the criteria weights in Bayesian Weight Analyzer analyses.

In [ ]:
weight_visualizer.vertical_jakplot(font_size=12, font_bold=True, ylim=[0, .4])